# Ordering and Delivery

This notebook demonstrates ordering and download with the orders api. In this notebook, we check authentication by requesting an orders list, then we create an order for two Analytic `PSScene4Band` images. We poll for order success then download images individually. And finally, we create, poll, and download the same order delivered as a single zip file.

Reference information can be found at [Ordering & Delivery](https://developers.planet.com/docs/orders/ordering-delivery/).

In [1]:
import json
import os
import pathlib
import time

import requests
from requests.auth import HTTPBasicAuth

## Authenticating

In [2]:
# API Key 
PLANET_API_KEY = "PLAKafb7581a5477426f8268729f7666e245"

In [3]:
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
data_url = "https://api.planet.com/data/v1"

### Curl example

To check your orders list and make sure you have the permissions you need, uncomment the following line to run `curl`

In [4]:
!curl -L -H "Authorization: api-key $PLANET_API_KEY" $orders_url

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2","next":"https://api.planet.com/compute/ops/orders/v2?page_marker=2023-04-27T16%3A35%3A18.030652Z"},"orders":[{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/bf7f0093-531e-4b2c-aecc-eee71081948b"},"created_on":"2023-05-12T20:32:20.160Z","delivery":{"archive_filename":"{{name}}_psscene_analytic_8b_sr_udm2.zip","archive_type":"zip","single_archive":true},"error_hints":[],"id":"bf7f0093-531e-4b2c-aecc-eee71081948b","last_message":"Manifest delivery completed","last_modified":"2023-05-12T20:51:10.452Z","name":"wuhan_cherry_download","notifications":{"email":true},"order_type":"partial","products":[{"item_ids":["20230306_024457_47_227b","20230306_024459_70_227b","20230313_020018_16_242b","20230314_024306_45_247f","20230326_020302_22_2430","20230326_020304_40_2430","20230326_024105_59_249c","20230401_020519_28_2465","20230401_020521_48_2465","20230407_021215_13_24b9","20230407_024359_07_247a"],"item_type":"PSSc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 29484    0 29484    0     0   197k      0 --:--:-- --:--:-- --:--:--  198k


In [5]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(data_url, auth=auth)
response

<Response [200]>

## Searching with the Data API
We can use the [data API](https://developers.planet.com/docs/apis/data/) in order to automate searching based on the search criterias like: date range, cloud cover, area cover, aoi.

In [6]:
geometry = {
    "type":"Polygon",
 "coordinates": [
          [
            [
              -119.49985069604644,
              34.06751955054469
            ],
            [
              -119.70691142646896,
              34.093031758032936
            ],
            [
              -120.00124568789624,
              34.11074404638616
            ],
            [
              -119.95589767668835,
              34.02781862205468
            ],
            [
              -119.89685969983209,
              33.936294004915155
            ],
            [
              -119.71375640929314,
              33.92067532189574
            ],
            [
              -119.51781877595889,
              33.96823238877933
            ],
            [
              -119.49985069604644,
              34.06751955054469
            ]
          ]
        ], "type":"Polygon"
}

In [7]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geometry
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte":"2020-01-01T00:00:00Z",
    "lte":"2020-02-01T00:00:00Z"
  }
}

# only get images which have <20% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.2
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [8]:
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(image_ids)

['20200131_182200_1006', '20200131_182159_1006', '20200131_182319_0f28', '20200131_182158_1006', '20200131_182318_0f28', '20200131_182316_0f28', '20200131_182317_0f28', '20200131_164619_0f49', '20200131_164617_0f49', '20200131_164618_0f49', '20200131_164616_0f49', '20200131_164617_1_0f49', '20200129_182251_0f35', '20200129_182250_0f35', '20200129_182248_0f35', '20200129_182249_0f35', '20200130_164751_1048', '20200130_164749_1_1048', '20200130_164750_1048', '20200130_164752_1048', '20200129_164801_0f3c', '20200129_164802_0f3c', '20200129_164800_0f3c', '20200129_182126_1029', '20200129_182127_1029', '20200129_182125_1029', '20200129_182124_1029', '20200130_182018_100a', '20200130_182017_100a', '20200129_164759_1_0f3c', '20200129_164759_0f3c', '20200130_182016_100a', '20200128_182228_1008', '20200128_182227_1008', '20200128_182226_1008', '20200128_181053_0e26', '20200128_181051_0e26', '20200127_182321_0f28', '20200127_182320_0f28', '20200127_182319_0f28', '20200126_164639_100d', '20200125

In [9]:
len(image_ids)

85

In [10]:
search_result.json()

{'_links': {'_first': 'https://api.planet.com/data/v1/searches/e8f25a3821b140a7b4dcd7221e018c88/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D',
  '_next': 'https://api.planet.com/data/v1/searches/e8f25a3821b140a7b4dcd7221e018c88/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIxLTAzLTEyVDE1OjIyOjU0LjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjAwMTAyXzE2NTIzMl8xMDRiIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319',
  '_self': 'https://api.planet.com/data/v1/searches/e8f25a3821b140a7b4dcd7221e018c88/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D'},
 'features': [{'_links': {'_self': 'https://ap

### Requests example

In this notebook, we will be using `requests` to communicate with the orders v2 API. First, we will check our orders list to make sure authentication and communication is working as expected.

We want to get a response code of `200` from this API call. To troubleshoot other response codes, see the [List Orders](https://developers.planet.com/docs/orders/reference/#operation/listOrders) AOI reference.

In [11]:
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

## Ordering

In this example, we will order two `PSScene4Band` analytic images. For variations on this kind of order, see [Ordering Data](https://developers.planet.com/docs/orders/ordering-delivery/#ordering-data_1).

In this order, we request an `analytic` bundle. A bundle is a group of assets for an item. The `analytic` bundle for the  `PSScene4Band` item contains 3 assets: the analytic image, the analytic xml file, and the udm. See the [Product bundles reference](https://developers.planet.com/docs/orders/product-bundles-reference/) to learn about other bundles and other items.

Now we will list the names of orders we have created thus far. Your list may be empty if you have not created an order yet.

In [12]:
orders = response.json()['orders']
[r['name'] for r in orders]

['wuhan_cherry_download',
 'cherries',
 'folder/7a13ca6c-de47-4b3b-bd68-7ab34a7dc198/SuperBloom',
 'folder/90e9cdf7-b77e-4ffb-8fc8-f94075d39d68/UCSBcampus',
 'West Campus 3m winter',
 'West Campus 3m winter',
 'WestCampus',
 'GreaterCampus2022',
 'UCSBcampus2022a',
 'UCSB2022',
 'UCSB2022',
 'folder/90e9cdf7-b77e-4ffb-8fc8-f94075d39d68/UCSBcampus',
 'folder/90e9cdf7-b77e-4ffb-8fc8-f94075d39d68/UCSBcampus',
 'campus3',
 'campus2',
 'campus 1',
 'carizzo',
 'carizzo',
 'folder/38398602-6ab7-47c8-9820-0ebd0cd33f71/carrizo plain',
 'folder/38398602-6ab7-47c8-9820-0ebd0cd33f71/carrizo plain']

### Place Order

In [13]:
# set content type to json
headers = {'content-type': 'application/json'}

In [14]:
# define products part of order
single_product = [
    {
      "item_ids": ["20220614_180918_90_2438"],
      "item_type": "PSScene",
      "product_bundle": "analytic_8b_sr_udm2"
    }
]

same_src_products = [
    {
      "item_ids": ["20151119_025740_0c74",
                   "20151119_025739_0c74"],
      "item_type": "PSScene4Band",
      "product_bundle": "analytic"
    }
]

multi_src_products = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene4Band",
      "product_bundle": "analytic"
    },
    {
      "item_ids": ["20220628_183020_20_248c"],
      "item_type": "PSScene",
      "product_bundle": "analytic_8b_udm2"
    },
    
]

In [23]:
product = [
    {
      "item_ids":image_ids,
      "item_type": "PSScene",
      "product_bundle": "visual"
    }
]


request = {  
   "name":"SantaCruzIsland2020_01",
   "products": product,
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [24]:
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response.json())
    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

In [25]:
order_url = place_order(request, auth)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/8e5af437-783b-449e-801a-e4e286a20295'}, 'created_on': '2023-05-19T18:00:34.007Z', 'delivery': {'archive_filename': 'output.zip', 'archive_type': 'zip', 'single_archive': True}, 'error_hints': [], 'id': '8e5af437-783b-449e-801a-e4e286a20295', 'last_message': 'Preparing order', 'last_modified': '2023-05-19T18:00:34.007Z', 'name': 'SantaCruzIsland2020_01', 'products': [{'item_ids': ['20200131_182200_1006', '20200131_182159_1006', '20200131_182319_0f28', '20200131_182158_1006', '20200131_182318_0f28', '20200131_182316_0f28', '20200131_182317_0f28', '20200131_164619_0f49', '20200131_164617_0f49', '20200131_164618_0f49', '20200131_164616_0f49', '20200131_164617_1_0f49', '20200129_182251_0f35', '20200129_182250_0f35', '20200129_182248_0f35', '20200129_182249_0f35', '20200130_164751_1048', '20200130_164749_1_1048', '20200130_164750_1048', '20200130_164752_1048', '20200129_164801_0f3c', '20200129_164802_0f3c', '20200129_164800_0

### Poll for Order Success

In [ ]:
def poll_for_success(order_url, auth, num_loops=5):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(order_url, auth)

## Bandmath and clip
Now lets use the toolchain in order to combine a few opperations at once.

To create a polygon to clip by lets go to [geojson.io](https://geojson.io)


In [ ]:
bandmath = {
  "bandmath": {
    "b1": ,
    "b2": ,
    "b3": ,
    "pixel_type": "32R",
  }
}

In [ ]:
tool_request = { 
    "name":"toolchain order",
    "products": product,
    "tools": [clip, bandmath],
    "delivery": {"single_archive": True, "archive_type": "zip"}
}

In [ ]:
tool_order_url = place_order(tool_request, auth)

### View Results
Now lets review our previous order and download it

In [ ]:
requests.get(order_url, auth=auth).json()['state']

In [ ]:
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']

In [ ]:
[r['name'] for r in results]

## Download

### Downloading each asset individually

In [ ]:
def download_results(results, overwrite=False):
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name in zip(results_urls, results_names):
        path = pathlib.Path(os.path.join('data', name))
        
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))

In [ ]:
download_results(results)

## Basemaps

In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
auth = HTTPBasicAuth(PLANET_API_KEY, '')
basemapServiceResponse = requests.get(url=BASEMAP_API_URL, auth=auth)

In [ ]:
basemaps = basemapServiceResponse.raise_for_status()
if basemapServiceResponse.status_code != 204:
    basemaps = json.loads(basemapServiceResponse.text)
mosaics = basemaps['mosaics']
names = [feature['name'] for feature in mosaics]
print(names)

In [ ]:
# set content type to json
headers = {'content-type': 'application/json'}

In [ ]:
# define the clip tool
clip = {
    "clip":{}
}

In [ ]:
order_params = {
   "name": "Basemap order with geometry",
   "source_type": "basemaps",
   "products": [
      {
         "mosaic_name": "global_quarterly_2023q1_mosaic",
         "geometry":geometry
      }
   ],
    "tools": [clip],
}

In [ ]:
basemap_url = place_order(order_params, auth)